# Kaggle Competition

You are provided with daily historical sales data. The task is to forecast the total amount of products sold in every shop for the test set. Note that the list of shops and products slightly changes every month. Creating a robust model that can handle such situations is part of the challenge.

### File descriptions
- sales_train.csv - the training set. Daily historical data from January 2013 to October 2015.
- test.csv - the test set. You need to forecast the sales for these shops and products for November 2015.



Link - https://www.kaggle.com/competitions/competitive-data-science-predict-future-sales/data <br/>
Git Link - https://github.com/Neelam-Singhal/Forecast_Sales_Kaggle_Competiton

In [1]:
import pandas as pd
import datetime
pd.set_option('display.float_format', lambda x: '%.3f' % x)


In [2]:
sales_train = pd.read_csv('sales_train.csv')
items = pd.read_csv('items.csv')
item_categories = pd.read_csv('item_categories.csv')
shops = pd.read_csv('shops.csv')

## Test
final_test = pd.read_csv('test.csv')
sample_submission = pd.read_csv('sample_submission.csv')

In [3]:
final_test.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [4]:
sample_submission.head()

,ID,item_cnt_month
0,0,0.500
1,1,0.500
2,2,0.500
3,3,0.500
4,4,0.500


### Data Overview
1. [NIU] Shops -> total 60 shops
2. [NIU] Item_categories -> Total 84 item categories
3. [NIU after merging] Items -> Has item_id and item_category_id. Can be merged with Sales DF to get item_category

In [5]:
#Merging item cat id with sales data
sales_train = sales_train.merge(items[['item_id', 'item_category_id']], on='item_id')
sales_train = sales_train[['date', 'date_block_num', 'shop_id', 'item_category_id', 'item_id', 'item_price', 'item_cnt_day']]
sales_train.date = pd.to_datetime(sales_train.date)

# EDA

In [6]:
df = sales_train.copy()


In [7]:
# Data Clean
df = df[df.date.between('2013-01-01', '2015-10-31') ]
df['month'] = df['date'].dt.month
df['year'] = df['date'].dt.year
df = df.sort_values('date')

In [16]:
df.sort_values(['date_block_num', 'shop_id'])

,date,date_block_num,shop_id,item_category_id,item_id,item_price,item_cnt_day,month,year
759068,2013-01-13,0,0,40,8587,77.000,1.000,1,2013
1093907,2013-01-13,0,0,40,12586,76.000,1.000,1,2013
504886,2013-01-13,0,0,6,11305,263.000,1.000,1,2013
256702,2013-01-13,0,0,21,3698,483.000,2.000,1,2013
593172,2013-01-13,0,0,40,14066,69.000,1.000,1,2013
...,...,...,...,...,...,...,...,...,...
2383367,2015-10-31,33,59,19,4872,1499.000,1.000,10,2015
2933891,2015-10-31,33,59,20,17755,1799.000,1.000,10,2015
2272261,2015-10-31,33,59,30,6687,199.000,1.000,10,2015
212799,2015-10-31,33,59,30,5811,98.000,1.000,10,2015


In [15]:
df.groupby(['date_block_num', 'shop_id', 'item_id']).agg({
    'item_category_id' : 'last', 
    'item_price':'sum',
    'item_cnt_day':'sum'
    
})

item_category_id  item_price  item_cnt_day
date_block_num shop_id item_id                                            
0              0       32                     40     884.000         6.000
                       33                     37    1041.000         3.000
                       35                     40     247.000         1.000
                       43                     40     221.000         1.000
                       51                     57     257.000         2.000
...                                          ...         ...           ...
33             59      22087                  83     357.000         6.000
                       22088                  83     238.000         2.000
                       22091                  83     179.000         1.000
                       22100                  42     629.000         1.000
                       22102                  42    1250.000         1.000

[1593410 rows x 3 columns]

In [15]:
df

,date,date_block_num,shop_id,item_category_id,item_id,item_price,item_cnt_day,month,year
13565,2013-01-01,0,28,30,2808,999.0,2.0,1,2013
208526,2013-01-01,0,54,2,5574,699.0,1.0,1,2013
208486,2013-01-01,0,27,2,5574,699.0,2.0,1,2013
208472,2013-01-01,0,28,2,5574,699.0,1.0,1,2013
485711,2013-01-01,0,13,40,17862,236.0,1.0,1,2013
...,...,...,...,...,...,...,...,...,...
900569,2015-10-31,33,42,37,15552,949.0,1.0,10,2015
2932863,2015-10-31,33,6,37,9928,799.0,1.0,10,2015
1923154,2015-10-31,33,48,30,7202,199.0,1.0,10,2015
2842285,2015-10-31,33,58,37,17388,799.0,1.0,10,2015


In [13]:
final_test

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268
...,...,...,...
214195,214195,45,18454
214196,214196,45,16188
214197,214197,45,15757
214198,214198,45,19648


In [104]:
df[(df.shop_id == 5) & (df.item_id == 4806)].sort_values(['date', 'shop_id', 'item_category_id','item_id'])

,date,date_block_num,shop_id,item_category_id,item_id,item_price,item_cnt_day,month,year
2174447,2014-01-06,17,5,30,4806,49.0,2.0,1,2014
2173826,2014-01-25,12,5,30,4806,299.0,1.0,1,2014
2174448,2014-02-06,17,5,30,4806,49.0,1.0,2,2014
2174911,2014-02-11,22,5,30,4806,299.0,1.0,2,2014
2173901,2014-02-17,13,5,30,4806,149.5,1.0,2,2014
...,...,...,...,...,...,...,...,...,...
2175853,2015-10-04,27,5,30,4806,99.0,1.0,10,2015
2176442,2015-10-08,31,5,30,4806,99.0,1.0,10,2015
2176681,2015-10-20,33,5,30,4806,99.0,1.0,10,2015
2176680,2015-10-24,33,5,30,4806,99.0,1.0,10,2015


In [105]:
df.date.min()

Timestamp('2013-01-01 00:00:00')

Timestamp('2015-10-31 00:00:00')